In [1]:
def parse_three_address_code(input_text):

    blocks = {}
    current_block_id = None
    instruction_counter = 1

    lines = input_text.strip().split('\n')
    for line in lines:
        line = line.strip()
        if not line:
            continue

        if line.startswith("Block"):
            try:
                current_block_id = int(line.split()[1])
                blocks[current_block_id] = []
            except (IndexError, ValueError):
                print(f"Warning: Invalid block header: {line}")
                continue
        elif current_block_id is not None:
            blocks[current_block_id].append((instruction_counter, line))
            instruction_counter += 1

    return blocks

def build_control_flow_graph(blocks):

    cfg = {}

    for block_id in blocks:
        cfg[block_id] = {'succ': set(), 'pred': set()}

    for block_id, instructions in blocks.items():
        if not instructions:
            continue

        last_instr = instructions[-1][1]

        if last_instr.startswith('if') and 'goto' in last_instr:
            try:
                target_parts = last_instr.split('goto')
                if len(target_parts) > 1:
                    target_block = int(target_parts[1].strip().split()[1])
                    cfg[block_id]['succ'].add(target_block)
                    cfg[target_block]['pred'].add(block_id)

                    next_block = block_id + 1
                    if next_block in blocks:
                        cfg[block_id]['succ'].add(next_block)
                        cfg[next_block]['pred'].add(block_id)
            except (IndexError, ValueError) as e:
                print(f"Warning: Error parsing branch in block {block_id}: {e}")

        elif last_instr.startswith('goto'):
            try:
                target_block = int(last_instr.split()[1].strip())
                cfg[block_id]['succ'].add(target_block)
                cfg[target_block]['pred'].add(block_id)
            except (IndexError, ValueError) as e:
                print(f"Warning: Error parsing jump in block {block_id}: {e}")

        elif block_id + 1 in blocks:
            cfg[block_id]['succ'].add(block_id + 1)
            cfg[block_id + 1]['pred'].add(block_id)

    return cfg

def compute_gen_kill(blocks):

    gen_kill = {}

    for block_id, instructions in blocks.items():
        gen_set = set()
        kill_set = set()
        defined_in_block = set()

        for instruction_no, instruction in instructions:
            parts = instruction.split()

            if len(parts) < 3 or parts[1] != '=' or instruction.startswith(('if', 'goto')):
                continue

            target = parts[0]
            defined_in_block.add(target)
            kill_set.add(instruction_no)

            for part in parts[2:]:
                if not part.isdigit() and part not in ['+', '-', '*', '/', '%', '(', ')', ',', '>=', '<=', '<', '>', '==']:

                    if part not in defined_in_block:
                        gen_set.add(instruction_no)

        gen_kill[block_id] = {'GEN': gen_set, 'KILL': kill_set}

    return gen_kill

def perform_in_out_analysis(blocks, cfg, gen_kill):

    analysis = {}
    for block_id in blocks:
        analysis[block_id] = {'IN': set(), 'OUT': set()}

    iteration = 1
    changed = True

    while changed:
        changed = False
        print(f"\nIteration {iteration}:")

        for block_id in sorted(blocks.keys()):
            old_in = analysis[block_id]['IN'].copy()
            old_out = analysis[block_id]['OUT'].copy()

            # Compute IN[i] = ∪ OUT[j] for all predecessors j of i
            in_set = set()
            for pred in cfg[block_id]['pred']:
                in_set = in_set.union(analysis[pred]['OUT'])
            analysis[block_id]['IN'] = in_set

            # Compute OUT[i] = GEN[i] ∪ (IN[i] - KILL[i])
            out_set = gen_kill[block_id]['GEN'].union(
                analysis[block_id]['IN'] - gen_kill[block_id]['KILL']
            )
            analysis[block_id]['OUT'] = out_set

            print(f"  Block {block_id}:")
            print(f"    IN:  {', '.join(map(str, sorted(analysis[block_id]['IN']))) if analysis[block_id]['IN'] else 'Ø'}")
            print(f"    OUT: {', '.join(map(str, sorted(analysis[block_id]['OUT']))) if analysis[block_id]['OUT'] else 'Ø'}")

            if old_in != analysis[block_id]['IN'] or old_out != analysis[block_id]['OUT']:
                changed = True

        iteration += 1

    for block_id in blocks:
        analysis[block_id]['GEN'] = gen_kill[block_id]['GEN']
        analysis[block_id]['KILL'] = gen_kill[block_id]['KILL']

    return analysis

def detect_loop_invariants(cfg, blocks):

    loop_invariants = {}

    for block_id in blocks:
        for succ in cfg[block_id]['succ']:
            if succ in cfg[block_id]['pred']:
                print(f"\nFound loop between Block {block_id} and Block {succ}")
                loop_invariants[block_id] = set()
                for instruction_no, instruction in blocks[block_id]:
                    parts = instruction.split()
                    print(f"  Analyzing instruction {instruction_no}: {instruction}")

                    if len(parts) >= 3 and parts[1] == '=':
                        lhs = parts[0]
                        rhs = parts[2:]

                        is_invariant = True
                        for var in rhs:
                            if var in ['i', 'n']:
                                is_invariant = False
                                break

                        if is_invariant:
                            print(f"    Loop Invariant detected: {lhs} = {''.join(rhs)}")
                            loop_invariants[block_id].add(lhs)
                        else:
                            print(f"    {lhs} is not a loop invariant because it depends on variables that change in the loop.")

    return loop_invariants




def main():
    print("Three Address Code Analysis with Blocks, Instruction Sets, and Loop Invariants")
    print("Enter your code blocks (format: 'Block X' followed by instructions, enter 'done' when finished):")

    input_lines = []
    while True:
        line = input()
        if line.lower() == 'done':
            break
        input_lines.append(line)

    input_text = '\n'.join(input_lines)
    blocks = parse_three_address_code(input_text)

    print("\nParsed Blocks:")
    for block_id in sorted(blocks.keys()):
        print(f"Block {block_id}:")
        for instruction_no, instruction in blocks[block_id]:
            print(f"  Instruction {instruction_no}: {instruction}")

    cfg = build_control_flow_graph(blocks)

    gen_kill = compute_gen_kill(blocks)
    analysis = perform_in_out_analysis(blocks, cfg, gen_kill)

    loop_invariants = detect_loop_invariants(cfg, blocks)

    print("\nFinal IN-OUT Analysis:")
    for block_id in sorted(analysis.keys()):
        print(f"Block {block_id}:")
        print(f"  GEN:  {', '.join(map(str, sorted(analysis[block_id]['GEN']))) if analysis[block_id]['GEN'] else 'Ø'}")
        print(f"  KILL: {', '.join(map(str, sorted(analysis[block_id]['KILL']))) if analysis[block_id]['KILL'] else 'Ø'}")
        print(f"  IN:   {', '.join(map(str, sorted(analysis[block_id]['IN']))) if analysis[block_id]['IN'] else 'Ø'}")
        print(f"  OUT:  {', '.join(map(str, sorted(analysis[block_id]['OUT']))) if analysis[block_id]['OUT'] else 'Ø'}")

    print("\nDetected Loop Invariants: Ø")
    for block_id, invariants in loop_invariants.items():
        print(f"Block {block_id}: Loop Invariants - {', '.join(map(str, sorted(invariants)))}")

if __name__ == "__main__":
    main()

Three Address Code Analysis with Blocks, Instruction Sets, and Loop Invariants
Enter your code blocks (format: 'Block X' followed by instructions, enter 'done' when finished):
Block 1
i = 0
sum = 0

Block 2
t1 = 4 * i
t2 = i * 10
if i >= 10 goto Block 4

Block 3
sum = sum + t2
i = i + 1
goto Block 2

 Block 4
return

Done

Parsed Blocks:
Block 1:
  Instruction 1: i = 0
  Instruction 2: sum = 0
Block 2:
  Instruction 3: t1 = 4 * i
  Instruction 4: t2 = i * 10
  Instruction 5: if i >= 10 goto Block 4
Block 3:
  Instruction 6: sum = sum + t2
  Instruction 7: i = i + 1
  Instruction 8: goto Block 2
Block 4:
  Instruction 9: return

Iteration 1:
  Block 1:
    IN:  Ø
    OUT: Ø
  Block 2:
    IN:  Ø
    OUT: 3, 4
  Block 3:
    IN:  3, 4
    OUT: 3, 4, 6
  Block 4:
    IN:  3, 4
    OUT: 3, 4

Iteration 2:
  Block 1:
    IN:  Ø
    OUT: Ø
  Block 2:
    IN:  Ø
    OUT: 3, 4
  Block 3:
    IN:  3, 4
    OUT: 3, 4, 6
  Block 4:
    IN:  3, 4
    OUT: 3, 4

Final IN-OUT Analysis:
Block 1:
  GE